In [187]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pyautogui
from selenium.webdriver.common.keys import Keys
import time
import json
import pandas as pd
import re
import csv
from selenium.common.exceptions import NoSuchElementException
import requests

In [188]:
# Чтение списка app_id из файла Excel
df_app_list = pd.read_excel("app_list.xlsx")
app_ids = df_app_list.iloc[:, 0].tolist() 

app_ids_list = []
qvd_filenames_list = []

In [189]:
# Инициализация браузера
driver = webdriver.Chrome()

# Загрузка страницы
driver.get("http://qlik.tattelecom.ru")

# Ожидание загрузки страницы
time.sleep(1)


Exception managing chrome: error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json): error trying to connect: tls handshake eof


In [190]:
# Вводим логин
pyautogui.write('tattelecom\\qlikdevelop')
pyautogui.press('tab')
time.sleep(0.5)

# Вводим пароль
pyautogui.write('q*4rt8kN%y')
pyautogui.press('enter')

# Подождать загрузки следующей страницы
time.sleep(1)

In [191]:
# Перебор всех app_id
for app_id in app_ids:
    url = f'http://qlik.tattelecom.ru/api/v1/apps/{app_id}/data/lineage'

    try:
        # Загрузка новой страницы для каждого app_id
        driver.get(url)
    
        # Ожидание загрузки страницы
        time.sleep(1.5)
    
        # Найти элемент <pre> с JSON текстом
        json_element = driver.find_element(By.XPATH, "//pre")

        # Получить текст из элемента <pre>
        json_text = json_element.text

        # Разбор JSON данных
        data = json.loads(json_text)

        # Перебор каждого элемента в JSON данных
        for item in data:
            discriminator = item.get("discriminator", "")
            qvd_matches = re.findall(r'\b\w+\.qvd\b', discriminator)

            for qvd_filename in qvd_matches:
                app_ids_list.append(app_id)
                qvd_filenames_list.append(qvd_filename)
    except NoSuchElementException:
        print(f"Element not found for app_id: {app_id}. Skipping to the next app_id.")

    except requests.exceptions.HTTPError as err:
        print(f"HTTP Error occurred for app_id: {app_id}. Error: {err}")
        continue
# Создаем датафрейм из списка данных all_data
df = pd.DataFrame({'app_id': app_ids_list, 'qvd_filename': qvd_filenames_list})

Element not found for app_id: ea79bab8-7140-4329-abcc-2cf34b22c236. Skipping to the next app_id.
Element not found for app_id: 4d4edfbc-2344-4c30-90ef-c3be17b3205d. Skipping to the next app_id.
Element not found for app_id: e6269a6e-76cb-446f-ae5f-1ef70cf2071c. Skipping to the next app_id.
Element not found for app_id: 0d3dbf22-f793-4566-bbbc-1b85d30cc47b. Skipping to the next app_id.
Element not found for app_id: 93e9041d-17fe-4c47-aadd-925261b78856. Skipping to the next app_id.
Element not found for app_id: 9c693a07-59c0-4a5d-a1d0-995b3c098606. Skipping to the next app_id.
Element not found for app_id: 4504144e-d97f-4206-a5e0-fc6b40ab2e45. Skipping to the next app_id.
Element not found for app_id: 19994d2c-8c15-4381-95cb-c70f551d5368. Skipping to the next app_id.


In [192]:
# Выводим датафрейм
print(df)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [193]:
df.to_csv('output.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)

#Закрыть браузер
driver.quit()